In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Final Project').master('spark://spark-master:7077').getOrCreate()
#spark = SparkSession.builder.appName('Final Project').master('local(*)').getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
64,application_1618931654412_0066,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
import pyspark.sql.functions as fn
from pyspark.sql.types import StructType, StructField, StringType,IntegerType
import pandas as pd
import numpy as np
import pyspark.ml.evaluation as ev
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
import pyspark.sql.types as types
import pyspark.ml.feature as ft
import pyspark.ml.classification as cl
from pyspark.ml import PipelineModel
import pyspark.ml.tuning as tune
from pyspark.sql.functions import isnan, when, count, col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
T4 = spark.read.csv('s3://msbx5420-spr21/sherlock/T4_filtered.tsv', sep=r'\t', header=False,inferSchema = True).toDF('UserId','UUID','t4_version','CpuHertz','CPU_0','CPU_1','CPU_2','CPU_3','Total_CPU',
               'TotalMemory_freeSize','TotalMemory_max_size','TotalMemory_total_size','TotalMemory_used_size',
               'Traffic_MobileRxBytes','Traffic_MobileRxPackets','Traffic_MobileTxBytes',
               'Traffic_MobileTxPackets','Traffic_TotalRxBytes','Traffic_TotalRxPackets','Traffic_TotalTxBytes',
               'Traffic_TotalTxPackets','Traffic_TotalWifiRxBytes','Traffic_TotalWifiRxPackets',
               'Traffic_TotalWifiTxBytes','Traffic_TotalWifiTxPackets','Traffic_timestamp','Battery_charge_type',
               'Battery_current_avg','Battery_health','Battery_icon_small','Battery_invalid_charger',
               'Battery_level','Battery_online','Battery_plugged','Battery_present','Battery_scale',
               'Battery_status','Battery_technology','Battery_temperature','Battery_timestamp','Battery_voltage',
               'MemTotal','MemFree','Buffers','Cached','SwapCached','Active','Inactive','Active_anon',
               'Inactive_anon','Active_file','Inactive_file','Unevictable','Mlocked','HighTotal','HighFree',
               'LowTotal','LowFree','SwapTotal','SwapFree','Dirty','Writeback','AnonPages','Mapped',
               'Shmem','Slab','SReclaimable','SUnreclaim','KernelStack','PageTables','CommitLimit','Committed_AS',
               'VmallocTotal','VmallocUsed','VmallocChunk','msmgpio_cpu0','msmgpio_sum_cpu123','wcd9xxx_cpu0',
               'wcd9xxx_sum_cpu123','pn547_cpu0','pn547_sum_cpu123','cypress_touchkey_cpu0',
               'cypress_touchkey_sum_cpu123','synaptics_rmi4_i2c_cpu0','synaptics_rmi4_i2c_sum_cpu123',
               'sec_headset_detect_cpu0','sec_headset_detect_sum_cpu123','flip_cover_cpu0','flip_cover_sum_cpu123',
               'home_key_cpu0','home_key_sum_cpu123','volume_down_cpu0','volume_down_sum_cpu123','volume_up_cpu0',
               'volume_up_sum_cpu123','companion_cpu0','companion_sum_cpu123','SLIMBUS_cpu0','SLIMBUS_sum_cpu123',
               'function_call_interrupts_cpu0','function_call_interrupts_sum_cpu123','cpu123_intr_prs','tot_user',
               'tot_nice','tot_system','tot_idle','tot_iowait','tot_irq','tot_softirq','ctxt','btime','processes',
               'procs_running','procs_blocked','connectedWifi_SSID','connectedWifi_Level', '117','118','119',
              '120', '121','122','123','124','125','126','127','128','129','130')

T4 = T4.withColumn("CPU_0",fn.col("CPU_0").cast('double')).withColumn("CPU_1",fn.col("CPU_1").cast('double')).withColumn("CPU_2",fn.col("CPU_2").cast('double')).withColumn("CPU_3",fn.col("CPU_3").cast('double'))
T4 = T4.fillna(value=0,subset=['CPU_0','CPU_1','CPU_2','CPU_3'])
cols = ['TotalMemory_freeSize','TotalMemory_max_size','TotalMemory_total_size','TotalMemory_used_size',
               'Traffic_MobileRxBytes','Traffic_MobileRxPackets','Traffic_MobileTxBytes',
               'Traffic_MobileTxPackets','Traffic_TotalRxBytes','Traffic_TotalRxPackets','Traffic_TotalTxBytes',
               'Traffic_TotalTxPackets','Traffic_TotalWifiRxBytes','Traffic_TotalWifiRxPackets',
               'Traffic_TotalWifiTxBytes','Traffic_TotalWifiTxPackets','Traffic_timestamp','Battery_charge_type',
               'Battery_current_avg','Battery_health','Battery_icon_small','Battery_invalid_charger',
               'Battery_level','Battery_online','Battery_plugged','Battery_present','Battery_scale',
               'Battery_status','Battery_technology','Battery_temperature','Battery_timestamp','Battery_voltage',
               'MemTotal','MemFree','Buffers','Cached','SwapCached','Active','Inactive','Active_anon',
               'Inactive_anon','Active_file','Inactive_file','Unevictable','Mlocked','HighTotal','HighFree',
               'LowTotal','LowFree','SwapTotal','SwapFree','Dirty','Writeback','AnonPages','Mapped',
               'Shmem','Slab','SReclaimable','SUnreclaim','KernelStack','PageTables','CommitLimit','Committed_AS',
               'VmallocTotal','VmallocUsed','VmallocChunk','msmgpio_cpu0','msmgpio_sum_cpu123','wcd9xxx_cpu0',
               'wcd9xxx_sum_cpu123','pn547_cpu0','pn547_sum_cpu123','cypress_touchkey_cpu0',
               'cypress_touchkey_sum_cpu123','synaptics_rmi4_i2c_cpu0','synaptics_rmi4_i2c_sum_cpu123',
               'sec_headset_detect_cpu0','sec_headset_detect_sum_cpu123','flip_cover_cpu0','flip_cover_sum_cpu123',
               'home_key_cpu0','home_key_sum_cpu123','volume_down_cpu0','volume_down_sum_cpu123','volume_up_cpu0',
               'volume_up_sum_cpu123','companion_cpu0','companion_sum_cpu123','SLIMBUS_cpu0','SLIMBUS_sum_cpu123',
               'function_call_interrupts_cpu0','function_call_interrupts_sum_cpu123','cpu123_intr_prs','tot_user',
               'tot_nice','tot_system','tot_idle','tot_iowait','tot_irq','tot_softirq','ctxt','btime','processes',
               'procs_running','procs_blocked','connectedWifi_SSID','connectedWifi_Level', '117','118','119',
              '120', '121','122','123','124','125','126','127','128','129','130']
T4 = T4.drop(*cols)

T4.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- UserId: string (nullable = true)
 |-- UUID: long (nullable = true)
 |-- t4_version: string (nullable = true)
 |-- CpuHertz: string (nullable = true)
 |-- CPU_0: double (nullable = false)
 |-- CPU_1: double (nullable = false)
 |-- CPU_2: double (nullable = false)
 |-- CPU_3: double (nullable = false)
 |-- Total_CPU: string (nullable = true)

In [ ]:
Moriarty = spark.read.options(inferSchema = True).csv('s3://msbx5420-spr21/sherlock/Moriarty.csv', header=True)
Moriarty = Moriarty.withColumnRenamed("Version","Version_Moriarty")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
Moriarty = Moriarty.select("*").toPandas()
Moriarty['ActionType_binary'] = np.where(Moriarty['ActionType']== 'malicious', 1, 0)
Moriarty['target'] = np.where(Moriarty['SessionType']== 'malicious', 1, 0)
Moriarty = Moriarty.dropna()
Moriarty = spark.createDataFrame(Moriarty)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
ScreenOn = spark.read.options(inferSchema = True).csv('s3://msbx5420-spr21/sherlock/ScreenOn.csv', header=True)

ScreenOn = ScreenOn.withColumn("Timestamp",fn.to_timestamp("Timestamp"))
ScreenOn = ScreenOn.withColumn("ScreenOn",fn.col("ScreenOn").cast("Integer"))
ScreenOn = ScreenOn.withColumnRenamed('Timestamp', 'Timestamp_ScreenOn').withColumnRenamed('UserID', 'UserId')

ScreenOn.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- UserId: string (nullable = true)
 |-- UUID: long (nullable = true)
 |-- ScreenOn: integer (nullable = true)
 |-- Timestamp_ScreenOn: timestamp (nullable = true)

## Join DF's 

In [ ]:
Moriarty.createOrReplaceTempView("Moriarty")
T4.createOrReplaceTempView("T4")
ScreenOn.createOrReplaceTempView("ScreenOn")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
ms4 = Moriarty.join(ScreenOn, ['UserId','UUID'],'full')
ms4 = ms4.join(T4, ['UserId','UUID'],'full')
ms4.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- UserId: string (nullable = true)
 |-- UUID: long (nullable = true)
 |-- Details: string (nullable = true)
 |-- Action: string (nullable = true)
 |-- ActionType: string (nullable = true)
 |-- SessionType: string (nullable = true)
 |-- Version_Moriarty: string (nullable = true)
 |-- SessionID: double (nullable = true)
 |-- ActionType_binary: long (nullable = true)
 |-- target: long (nullable = true)
 |-- ScreenOn: integer (nullable = true)
 |-- Timestamp_ScreenOn: timestamp (nullable = true)
 |-- t4_version: string (nullable = true)
 |-- CpuHertz: string (nullable = true)
 |-- CPU_0: double (nullable = true)
 |-- CPU_1: double (nullable = true)
 |-- CPU_2: double (nullable = true)
 |-- CPU_3: double (nullable = true)
 |-- Total_CPU: string (nullable = true)

In [ ]:
cols = ['Details','SessionType','ActionType','Version_Moriarty','SessionID','ActionType_binary','Timestamp_ScreenOn','t4_version','CpuHertz','Total_CPU']
ms4 = ms4.drop(*cols)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
ms4 = ms4.fillna(value=0,subset=["target"])
ms4 = ms4.fillna(value='None',subset=["Action"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# get the sort key
def getKey(item):
    return item.UUID

# fill function
def fill(x):
    out = []
    last_val_so = None
    last_val_0 = None
    last_val_1 = None
    last_val_2 = None
    last_val_3 = None
    for v in x:
        if v["ScreenOn"] is None:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], last_val_so, v["CPU_0"], v["CPU_1"], v["CPU_2"], v["CPU_3"]]
        else:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], v["ScreenOn"], v["CPU_0"], v["CPU_1"], v["CPU_2"], v["CPU_3"]]
            last_val_so = v["ScreenOn"]
        if v["CPU_0"] is None:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], v["ScreenOn"], last_val_0, v["CPU_1"], v["CPU_2"], v["CPU_3"]]
        else:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], v["ScreenOn"], v["CPU_0"], v["CPU_1"], v["CPU_2"], v["CPU_3"]]
            last_val_0 = v["CPU_0"]
        if v["CPU_1"] is None:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], v["ScreenOn"], v["CPU_0"], last_val_1, v["CPU_2"], v["CPU_3"]]
        else:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], v["ScreenOn"], v["CPU_0"], v["CPU_1"], v["CPU_2"], v["CPU_3"]]
            last_val_1 = v["CPU_1"]
        if v["CPU_2"] is None:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], v["ScreenOn"], v["CPU_0"], v["CPU_1"], last_val_2, v["CPU_3"]]
        else:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], v["ScreenOn"], v["CPU_0"], v["CPU_1"], v["CPU_2"], v["CPU_3"]]
            last_val_2 = v["CPU_2"]
        if v["CPU_3"] is None:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], v["ScreenOn"], v["CPU_0"], v["CPU_1"], v["CPU_2"], last_val_3]
        else:
            data = [v["UserId"], v["UUID"], v["Action"], v["target"], v["ScreenOn"], v["CPU_0"], v["CPU_1"], v["CPU_2"], v["CPU_3"]]
            last_val_3 = v["CPU_3"]
        out.append(data)
    return out

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Partition the data
rdd = ms4.rdd.groupBy(lambda x: x.UserId).mapValues(list)
# Sort the data by date
rdd = rdd.mapValues(lambda x: sorted(x, key=getKey))
# fill missing value and flatten
rdd = rdd.mapValues(fill).flatMapValues(lambda x: x)
# discard the key
rdd = rdd.map(lambda v: v[1])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType
schema = StructType([StructField("UserId", StringType(), True), StructField("UUID", LongType(), True), StructField("Action", StringType(), True), StructField("target", IntegerType(), True), StructField("ScreenOn", IntegerType(), True), StructField("CPU_0", DoubleType(), True), StructField("CPU_1", DoubleType(), True), StructField("CPU_2", DoubleType(), True), StructField("CPU_3", DoubleType(), True)])
ms4 = sqlContext.createDataFrame(rdd,schema)
ms4.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- UserId: string (nullable = true)
 |-- UUID: long (nullable = true)
 |-- Action: string (nullable = true)
 |-- target: integer (nullable = true)
 |-- ScreenOn: integer (nullable = true)
 |-- CPU_0: double (nullable = true)
 |-- CPU_1: double (nullable = true)
 |-- CPU_2: double (nullable = true)
 |-- CPU_3: double (nullable = true)

In [ ]:
ms4 = ms4.fillna(value=0,subset=["ScreenOn",'CPU_0','CPU_1','CPU_2','CPU_3'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
ms4.write.parquet('s3://msbx5420-spr21/sherlock/ms4_update.parquet')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
ms4 = spark.read.parquet('s3://msbx5420-spr21/sherlock/ms4_update.parquet', header=True, inferSchema=True)
ms4.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- UserId: string (nullable = true)
 |-- UUID: long (nullable = true)
 |-- Action: string (nullable = true)
 |-- target: integer (nullable = true)
 |-- ScreenOn: integer (nullable = true)
 |-- CPU_0: double (nullable = true)
 |-- CPU_1: double (nullable = true)
 |-- CPU_2: double (nullable = true)
 |-- CPU_3: double (nullable = true)

## Create Transformer

In [ ]:
ms4 = StringIndexer(inputCol='Action', outputCol="a_index").fit(ms4).transform(ms4)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
encoder = ft.OneHotEncoder(inputCol="a_index", outputCol='avec')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
featuresCreator = ft.VectorAssembler(inputCols=[col for col in ms4.columns[4:9]] + [encoder.getOutputCol()], outputCol='features')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create Estimator

In [ ]:
logistic = cl.LogisticRegression(maxIter=10, regParam=0.01, labelCol='target')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create Pipeline

In [ ]:
pipeline = Pipeline(stages=[encoder, featuresCreator, logistic])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Fit Model

In [ ]:
ms4_train, ms4_test = ms4.randomSplit([0.7, 0.3], seed=200)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
model = pipeline.fit(ms4_train)
test_model = model.transform(ms4_test)
test_model.take(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(UserId='0a50e09262', UUID=1458086406466, Action='None', target=0, ScreenOn=0, CPU_0=0.0, CPU_1=20.0, CPU_2=0.0, CPU_3=20.0, a_index=0.0, avec=SparseVector(25, {0: 1.0}), features=SparseVector(30, {2: 20.0, 4: 20.0, 5: 1.0}), rawPrediction=DenseVector([12.8765, -12.8765]), probability=DenseVector([1.0, 0.0]), prediction=0.0), Row(UserId='0a50e09262', UUID=1458086421857, Action='None', target=0, ScreenOn=0, CPU_0=50.0, CPU_1=28.57143, CPU_2=77.27273, CPU_3=10.526316, a_index=0.0, avec=SparseVector(25, {0: 1.0}), features=SparseVector(30, {1: 50.0, 2: 28.5714, 3: 77.2727, 4: 10.5263, 5: 1.0}), rawPrediction=DenseVector([12.8772, -12.8772]), probability=DenseVector([1.0, 0.0]), prediction=0.0), Row(UserId='0a50e09262', UUID=1458086446874, Action='None', target=0, ScreenOn=0, CPU_0=11.764706, CPU_1=0.0, CPU_2=0.0, CPU_3=65.0, a_index=0.0, avec=SparseVector(25, {0: 1.0}), features=SparseVector(30, {1: 11.7647, 4: 65.0, 5: 1.0}), rawPrediction=DenseVector([12.8767, -12.8767]), probabilit

## Model Performance 

In [ ]:
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='target')

print(evaluator.evaluate(test_model, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(test_model, {evaluator.metricName: 'areaUnderPR'}))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7987667354479582
0.0002033546505104903

## Saving Model

In [ ]:
pipeline_path = './Logistic_Pipeline'
pipeline.write().overwrite().save(pipeline_path)

model_path = './Logistic_Model'
model.write().overwrite().save(model_path)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Hyperparameter Tuning

In [ ]:
logistic = cl.LogisticRegression(labelCol='target')

grid = tune.ParamGridBuilder().addGrid(logistic.maxIter, [2, 10, 50]).addGrid(logistic.regParam, [0.01, 0.05, 0.3]).build()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
evaluator = ev.BinaryClassificationEvaluator(rawPredictionCol='probability', labelCol='target')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
cv = tune.CrossValidator(estimator=logistic, estimatorParamMaps=grid, evaluator=evaluator)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
pipeline = Pipeline(stages=[encoder, featuresCreator])
data_transformer = pipeline.fit(ms4_train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
cvModel = cv.fit(data_transformer.transform(ms4_train))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
data_test = data_transformer.transform(ms4_test)
results = cvModel.transform(data_test)

print(evaluator.evaluate(results, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(results, {evaluator.metricName: 'areaUnderPR'}))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.7987667354479582
0.0002033546505104903

In [ ]:
cvModel.getEstimatorParamMaps()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{Param(parent='LogisticRegression_0cca2bbc7a91', name='maxIter', doc='max number of iterations (>= 0).'): 2, Param(parent='LogisticRegression_0cca2bbc7a91', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='LogisticRegression_0cca2bbc7a91', name='maxIter', doc='max number of iterations (>= 0).'): 2, Param(parent='LogisticRegression_0cca2bbc7a91', name='regParam', doc='regularization parameter (>= 0).'): 0.05}, {Param(parent='LogisticRegression_0cca2bbc7a91', name='maxIter', doc='max number of iterations (>= 0).'): 2, Param(parent='LogisticRegression_0cca2bbc7a91', name='regParam', doc='regularization parameter (>= 0).'): 0.3}, {Param(parent='LogisticRegression_0cca2bbc7a91', name='maxIter', doc='max number of iterations (>= 0).'): 10, Param(parent='LogisticRegression_0cca2bbc7a91', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='LogisticRegression_0cca2bbc7a91', name='maxIter', doc='max number of iterations (>= 0).'): 10, P

In [ ]:
cvModel.avgMetrics

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.7991346419558265, 0.7991346419558265, 0.7991346419558265, 0.7991379654395625, 0.7991343398209414, 0.7991346419558265, 0.7991379654395625, 0.7991346419558265, 0.7991346419558265]

In [ ]:
results = []
for params, metric in zip(cvModel.getEstimatorParamMaps(), cvModel.avgMetrics):
    parameters = []
    for key, paramValue in zip(params.keys(), params.values()):
        parameters.append({key.name: paramValue})
    results.append((parameters, metric))

sorted(results, key=lambda x: x[1], reverse=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[([{'maxIter': 10}, {'regParam': 0.01}], 0.7991379654395625), ([{'maxIter': 50}, {'regParam': 0.01}], 0.7991379654395625), ([{'maxIter': 2}, {'regParam': 0.01}], 0.7991346419558265), ([{'maxIter': 2}, {'regParam': 0.05}], 0.7991346419558265), ([{'maxIter': 2}, {'regParam': 0.3}], 0.7991346419558265), ([{'maxIter': 10}, {'regParam': 0.3}], 0.7991346419558265), ([{'maxIter': 50}, {'regParam': 0.05}], 0.7991346419558265), ([{'maxIter': 50}, {'regParam': 0.3}], 0.7991346419558265), ([{'maxIter': 10}, {'regParam': 0.05}], 0.7991343398209414)]

## Train-Validation Splitting

In [ ]:
selector = ft.ChiSqSelector(numTopFeatures=5, featuresCol=featuresCreator.getOutputCol(), outputCol='selectedFeatures', labelCol='target')

logistic = cl.LogisticRegression(labelCol='target', featuresCol='selectedFeatures')

pipeline = Pipeline(stages=[encoder, featuresCreator, selector])
feature_transformer = pipeline.fit(ms4_train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
grid = tune.ParamGridBuilder().addGrid(logistic.maxIter, [2, 10, 50]).addGrid(logistic.regParam, [0.01, 0.05, 0.3]).build()
tvs = tune.TrainValidationSplit(estimator=logistic, estimatorParamMaps=grid, evaluator=evaluator, collectSubModels=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
tvsModel = tvs.fit(feature_transformer.transform(ms4_train))

data_test = feature_transformer.transform(ms4_test)
results = tvsModel.transform(data_test)

print(evaluator.evaluate(results, {evaluator.metricName: 'areaUnderROC'}))
print(evaluator.evaluate(results, {evaluator.metricName: 'areaUnderPR'}))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.0
1.0

In [ ]:
tvsModel.validationMetrics

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [ ]:
results = []
for params, metric in zip(tvsModel.getEstimatorParamMaps(), tvsModel.validationMetrics):
    parameters = []
    for key, paramValue in zip(params.keys(), params.values()):
        parameters.append({key.name: paramValue})
    results.append((parameters, metric))

sorted(results, key=lambda x: x[1], reverse=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[([{'maxIter': 2}, {'regParam': 0.01}], 1.0), ([{'maxIter': 2}, {'regParam': 0.05}], 1.0), ([{'maxIter': 2}, {'regParam': 0.3}], 1.0), ([{'maxIter': 10}, {'regParam': 0.01}], 1.0), ([{'maxIter': 10}, {'regParam': 0.05}], 1.0), ([{'maxIter': 10}, {'regParam': 0.3}], 1.0), ([{'maxIter': 50}, {'regParam': 0.01}], 1.0), ([{'maxIter': 50}, {'regParam': 0.05}], 1.0), ([{'maxIter': 50}, {'regParam': 0.3}], 1.0)]

## Random Forest

In [ ]:
ms4 = ms4.withColumn('target', fn.col('target').cast(types.DoubleType()))
ms4_train, ms4_test = ms4.randomSplit([0.7, 0.3], seed=200)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
classifier = cl.RandomForestClassifier(numTrees=5, maxDepth=5, labelCol='target')
pipeline = Pipeline(stages=[encoder, featuresCreator, classifier])
model = pipeline.fit(ms4_train)
test = model.transform(ms4_test)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
evaluator = ev.BinaryClassificationEvaluator(labelCol='target')

print(evaluator.evaluate(test, {evaluator.metricName: "areaUnderROC"}))
print(evaluator.evaluate(test, {evaluator.metricName: "areaUnderPR"}))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.0
1.0

In [ ]:
classifier = cl.DecisionTreeClassifier(maxDepth=5, labelCol='target')
pipeline = Pipeline(stages=[encoder, featuresCreator, classifier])
model = pipeline.fit(ms4_train)
test = model.transform(ms4_test)

evaluator = ev.BinaryClassificationEvaluator(labelCol='target')
print(evaluator.evaluate(test, {evaluator.metricName: "areaUnderROC"}))
print(evaluator.evaluate(test, {evaluator.metricName: "areaUnderPR"}))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1.0
1.0